In [1]:
import argparse
import cv2
import pandas as pd
import os
import re

In [2]:
ref = []

In [3]:
regex = re.compile(r'\\(\w*)$')

In [4]:
try:
    images_data = pd.read_csv('images.csv', delimiter=',')
except:
    images_data = None
images_list = []

In [5]:
def click_and_crop(event, x, y, flags, param):
    global ref
    global image
    drawing = False
    if event == cv2.EVENT_LBUTTONDOWN:
        ref.append((x, y))
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.rectangle(image, ref[0], [x, y], (255, 0, 0), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        ref.append((x, y))
    
    if len(ref) > 1:
        for i, (a, b) in enumerate(zip(ref, ref[1:])):
            if not i % 2:
                cv2.rectangle(image, a, b, (0, 255, 0), 2)
                cv2.imshow("image", image)
            else:
                continue
        

In [6]:
def one_image(root, image_name):
    global images_list
    global image
    global ref
    stop = False
    ref = []
    if images_data is not None:
        if images_data['image_name'].isin([image_name]).any():
            print('Image already in file')
            return False
    image_type = regex.findall(root)[0]
    image = cv2.imread(os.path.join(root, name))
    clone = image.copy()
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", click_and_crop)
    while True:
        cv2.imshow("image", image)
        key = cv2.waitKey(0) & 0XFF

        #reset crop if r is pressed
        if key == ord('r'):
            ref = []
            image = clone.copy()

        #break if c
        elif key == ord('c'):
            break
            
        elif key == ord('s'):
            stop = True
            break

    if len(ref) > 1:
        for i, (a, b) in enumerate(zip(ref, ref[1:])):
            if not i % 2:
                images_list.append({
                        'image_type': image_type,
                        'image_name': image_name,
                        'x1': a[0],
                        'y1': a[1],
                        'x2': b[0],
                        'y2': b[1]
                    })

    cv2.destroyAllWindows()
    return stop

In [7]:
all_images = os.walk(r'C:\Users\Alfonso\Documents\Scripts\Kaggle\train')
it = iter(all_images)
stop = False
for root, paths, names in os.walk(r'C:\Users\Alfonso\Documents\Scripts\Kaggle\train'):
    if stop:
        break
    print("Root: {}".format(root))
    if len(names) > 0:
        for name in names:
            print("Filepath: {}".format(os.path.join(root, name)))
            try:
                if not one_image(root, name):
                    continue
                else:
                    stop = True
                    break
            except:
                continue

if images_data is not None:
    new_data = pd.concat([images_data, pd.DataFrame(images_list)], axis=0, ignore_index=True)
    images_data.to_csv('new_results.csv', mode='w+')
    print('Data saved')
else:
    temp = pd.DataFrame(images_list)
    temp.to_csv('new_results.csv')
    print('New Data saved')

Root: C:\Users\Alfonso\Documents\Scripts\Kaggle\train
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\.DS_Store
Root: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB\img_00003.jpg
Image already in file
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB\img_00010.jpg
Image already in file
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB\img_00012.jpg
Image already in file
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB\img_00015.jpg
Filepath: C:\Users\Alfonso\Documents\Scripts\Kaggle\train\ALB\img_00019.jpg


ValueError: Plan shapes are not aligned